# Improts and setup

### Setup envirnonment 

In [1]:
%load_ext autoreload
%autoreload 2

### Import necessaary libraries

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image


import matplotlib as mpl
import matplotlib.pyplot as plt


import tempfile
from urllib.request import urlretrieve
import tarfile
import json
import imp
import os

import IPython.display as display
import PIL.Image


import numpy as np

### Import custom libraries

In [3]:
import patch_utils
import image_utils

In [8]:
imp.reload(patch_utils)
imp.reload(image_utils)

<module 'image_utils' from '/home/slavchic/Desktop/209AS_winter_2020/python_files/image_utils/__init__.py'>

# Workspace

### Load labels

In [9]:
imagenet_json, _ = urlretrieve(
    'http://www.anishathalye.com/media/2017/07/25/imagenet.json')
with open(imagenet_json) as f:
    imagenet_labels = json.load(f)### Choose an image

In [41]:
# specify training set
number_of_images = 5
my_classes = ['cat', 'ship']
my_valid_range = (0, 10)


# specify the patch size, mean, std
mean = 0
std = 0.3
size = 50

# get train images 
train_set = image_utils.sample_images(my_classes, number_of_images, valid_range = my_valid_range)

### Patch setup

In [65]:
# patch creator
class patch_multiplier(layers.Layer):
    
    def __init__(self, patch_size=50, patch_mean=0, patch_std=0.3):
        super(patch_multiplier, self).__init__()
        self.patch_size = patch_size
        self.patch_mean = patch_mean
        self.patch_std = patch_std
        
    def build(self, input_shape):
        self.patch = self.add_weight(shape=(self.patch_size, self.patch_size, 3),
                                     initializer=tf.random_normal_initializer(mean=self.patch_mean, 
                                                                              stddev=self.patch_std, 
                                                                              seed=None),
                                     trainable=True,
                                     name='patch')
        
    def call(self, inputs):
        multiple_patches = tf.broadcast_to(self.patch, (tf.shape(inputs)[0], self.patch_size, self.patch_size, 3))
        return multiple_patches



# patch shift layer
# Note: order matters. First argument is patches, second has shape=[x_shift, y_shift]
class patch_shift(layers.Layer):
    
    def __init__(self):
        super(patch_shift, self).__init__()
        
    def build(self, input_shape):
        pass
    
    def call(self, inputs):    
        # get dimensions of the patch
        dims = tf.shape(inputs[0])

        # update patch lenth
        a = tf.concat((inputs[0], -2 * tf.ones(shape=(dims[0], dims[1], 299 - dims[2], 3), dtype=tf.float32)), axis=2)

        # update patch height
        b = tf.concat((a, -2 * tf.ones(shape=(dims[0], 299 - dims[1], 299, 3))), axis=1)

        # shift the patch to the place needed
        mapped_patches = tf.roll(b, shift = (inputs[1][0][0], inputs[1][0][1]), axis = (2, 1))

        return mapped_patches

    
    
    
    
# patch application layer
# Note: order matters. First argument is images, second is patches
class patch_applicator(layers.Layer):
    
    def __init__(self):
        super(patch_applicator, self).__init__()
        
    def build(self, input_shape):
        pass
    
    def call(self, inputs):
        # prepare a mask for original images
        img_mask = tf.math.abs(tf.math.add(tf.math.sign(tf.math.add(inputs[1], 2)), -1))
        # crop the place for the pathch
        cropped_images = tf.math.multiply(inputs[0], img_mask)

        # prepare a mask for patches
        patch_mask = tf.math.sign(tf.math.add(inputs[1], 2))

        # crop patch from the field of -2-s
        cropped_patch = tf.math.multiply(inputs[1], patch_mask)

        # get images with patches
        images_with_patches = cropped_images + cropped_patch
        
        return images_with_patches

### Graph setup

In [139]:
# clear previous session
tf.keras.backend.clear_session() 


# initialize inputs
input_image = tf.keras.layers.Input(shape=(299, 299, 3))
input_shift = tf.keras.layers.Input(shape=(2,), dtype=tf.int32)


# multiply patch
patch_multiplier_layer = patch_multiplier(patch_size=size)
patch_array = patch_multiplier_layer(input_image)


# shift patch
patch_shift_layer = patch_shift()
shifted_patch_array = patch_shift_layer([patch_array, input_shift])


# apply patch
patch_applicator_layer = patch_applicator()
adv_images = patch_applicator_layer([input_image, shifted_patch_array])


# declare submodel for convenience
image_processing_module = keras.models.Model(inputs=[input_image, input_shift], outputs=adv_images)


# declare inception and set it parameters as non-trainable
incv3 = tf.keras.applications.InceptionV3(weights="imagenet")
incv3.trainable=False


# connect output of submodel to the input of inception
out = incv3(image_processing_module.output)


# declare main model
adv_learning_model = keras.models.Model(inputs=[input_image, input_shift], outputs=out)

In [140]:
adv_learning_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
patch_multiplier (patch_multipl (None, 50, 50, 3)    7500        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
patch_shift (patch_shift)       (None, None, 299, 3) 0           patch_multiplier[0][0]           
                                                                 input_2[0][0]              

In [135]:
# clear previous session
tf.keras.backend.clear_session() 

shifted_patch_array = None

# initialize inputs
input_image = tf.keras.layers.Input(shape=(299, 299, 3))
input_shift = tf.keras.layers.Input(shape=(2,), dtype=tf.int32)

print(input_image)

# multiply patch
patch_multiplier_layer = patch_multiplier(patch_size=size)
patch_array = patch_multiplier_layer(input_image)


# shift patch
patch_shift_layer = patch_shift()
shifted_patch_array = patch_shift_layer([patch_array, input_shift])

# apply patch
patch_applicator_layer = patch_applicator()
adv_images = patch_applicator_layer([input_image, shifted_patch_array])



incv3 = tf.keras.applications.InceptionV3(weights="imagenet")
incv3.trainable=False
x = incv3(adv_images)


model1 = keras.models.Model(inputs=[input_image, input_shift], outputs=x)


Tensor("input_1:0", shape=(None, 299, 299, 3), dtype=float32)


In [136]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
patch_multiplier (patch_multipl (None, 50, 50, 3)    7500        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
patch_shift (patch_shift)       (None, None, 299, 3) 0           patch_multiplier[0][0]           
                                                                 input_2[0][0]                